In [4]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

In [47]:
cap = cv2.VideoCapture(0)

ret, frame = cap.read()

face_cascade = cv2.CascadeClassifier('/home/ashu/Desktop/ComputerVision/Computer-Vision-with-Python/DATA/haarcascades/haarcascade_frontalface_default.xml')
face_rect = face_cascade.detectMultiScale(frame)


In [ ]:
(face_x, face_y, w, h) = tuple(face_rect[0])
tracking_window = (face_x, face_y, w, h)

roi = frame[face_y:face_y+h, face_x:face_x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])

cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 15)

while True:

    ret, frame = cap.read()

    if ret == True:

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)

        #mean shift method
        ret, tracking_window = cv2.meanShift(dst, tracking_window, term_crit)

        x,y,w,h = tracking_window
        img2 = cv2.rectangle(frame, (x,y), (x+w,y+w), (255,0,0), 10)

        cv2.imshow('Face tracking',img2)

        k = cv2.waitKey(1) & 0xFF
        if k==27:
            break
    
    else:
        break

cap.release()
cv2.destroyAllWindows()

In [ ]:
##camshift method
(face_x, face_y, w, h) = tuple(face_rect[0])
tracking_window = (face_x, face_y, w, h)

roi = frame[face_y:face_y+h, face_x:face_x+w]
hsv_roi = cv2.cvtColor(roi, cv2.COLOR_BGR2HSV)
roi_hist = cv2.calcHist([hsv_roi], [0], None, [180], [0,180])

cv2.normalize(roi_hist, roi_hist, 0, 255, cv2.NORM_MINMAX)

term_crit = (cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT, 5, 15)

while True:

    ret, frame = cap.read()

    if ret == True:

        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

        dst = cv2.calcBackProject([hsv], [0], roi_hist, [0,180], 1)

        #cam shift method
        ret, tracking_window = cv2.CamShift(dst, tracking_window, term_crit)
        pts = cv2.boxPoints(ret)
        pts = np.int0(pts)
        img2 = cv2.polylines(frame, [pts], True, (0,0,255), 10)

        ########################################################################
        cv2.imshow('Face tracking',img2)

        k = cv2.waitKey(1) & 0xFF
        if k==27:
            break
    
    else:
        break

cap.release()
cv2.destroyAllWindows()

Using Tracking APIs

In [ ]:
tracker = cv2.TrackerMIL_create()

cap = cv2.VideoCapture(0)

ret, frame = cap.read()

roi = cv2.selectROI(frame, False) #for drawing the desired region of interest

ret = tracker.init(frame, roi)

while True:

    ret, frame = cap.read()

    success, roi = tracker.update(frame)

    (x,y,w,h) = tuple(map(int,roi))

    if success:
        pt1 = (x,y)
        pt2 = (x+w,y+h)

        cv2.rectangle(frame, pt1, pt2, (255,0,0), 5)
    else:
        cv2.putText(frame, 'Failure to Detect object', (100,200), cv2.FONT_HERSHEY_DUPLEX, 10, (0,0,255), 10)
    
    cv2.imshow('Tracking', frame)

    k = cv2.waitKey(1) & 0xff
    if k==27:
        break

cap.release()
cv2.destroyAllWindows()